In [1]:
import numpy as np
from glob import glob
import pandas as pd
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import os
from shutil import copyfile
import seaborn as sea
from astropy.table import Table
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({"xtick.direction": "in","ytick.direction": "in"})
cmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.1, as_cmap=True)

In [2]:
def Remove_ns(gid):
    x = [int(U[1:]) for U in gid]
    return np.array(x)

def Get_rshift(table, ID):
    z = table['z_m1'][ID-1]
    return z

In [3]:
uvjDB = pd.read_pickle('../data/uvj_db.pkl')
quvjDB = uvjDB[uvjDB['Q/S'].map(lambda x:x.startswith('q'))]
ids = Remove_ns(quvjDB.index)
smask = np.array([U.startswith('s') for U in quvjDB.index])
nmask = np.array([U.startswith('n') for U in quvjDB.index])

goodss = '../../../catalogs_for_CLEAR/goodss_3dhst.v4.1.cats/Eazy/goodss_3dhst.v4.1.zout'
scat = Table.read(goodss,format='ascii')

goodsn = '../../../catalogs_for_CLEAR/goodsn_3dhst.v4.1.cats/Eazy/goodsn_3dhst.v4.1.zout'
ncat = Table.read(goodsn,format='ascii')

s_z = np.array([Get_rshift(scat,U) for U in ids[smask]])
n_z = np.array([Get_rshift(ncat,U) for U in ids[nmask]])
z_tot = np.append(n_z,s_z)

In [6]:
qgalDB = pd.DataFrame({'gids' : quvjDB.index,'ids' : ids, 'redshift':z_tot})
pd.to_pickle(qgalDB,'../data/quiescent_gal_DB.pkl')

In [5]:
qgalDB

,gids,ids,redshift
0,n542,542,1.213
1,n1184,1184,1.209
2,n2023,2023,1.680
3,n2116,2116,1.018
4,n2265,2265,1.104
5,n2302,2302,0.978
6,n2484,2484,1.051
7,n2684,2684,1.007
8,n2882,2882,1.139
9,n2913,2913,1.108


In [7]:
hd_path = '../../../../../Volumes/Vince_homedrive/Extractions/'

obv_dir =glob(hd_path + '*-*')

In [70]:
def Get_file_list(dirs,gid):
    field = gid[0]
    ID = int(gid[1:])
    direct = '../../../../../Volumes/Vince_homedrive/Extractions/Quiescent_galaxies/%s' % gid
       
    one_d =np.array([])
    one_d_n =np.array([])
    for i in range(len(dirs)):
        if field == 's':
            pres = glob(dirs[i] + '/1D/FITS/' + '*S*%s*' % ID)
        else:
            pres = glob(dirs[i] + '/1D/FITS/' + '*N*%s*' % ID)

        if len(pres) >=1:
            one_d= np.append(one_d,pres)
            pres_n = pres[0].replace(dirs[i] + '/1D/FITS/','')
            one_d_n= np.append(one_d_n,pres_n)
       
    if len(one_d) >=1:
        if field == 's':
            two_d =glob('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/2D/PNG/*S*%s*' % ID)
            if len(two_d) >= 1:
                two_d_n = []
                for i in range(len(two_d)):
                    two_d_n.append(two_d[i].replace('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/2D/PNG/',''))
        else:
            two_d =glob('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/2D/PNG/*N*%s*' % ID)
            if len(two_d) >= 1:
                two_d_n = []
                for i in range(len(two_d)):
                    two_d_n.append(two_d[i].replace('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/2D/PNG/',''))
                    
        if field == 's':
            spec =glob('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/1D/FITS/*S*%s*' % ID)
            if len(spec) >= 1:
                spec_n = []
                for i in range(len(spec)):
                    spec_n.append(spec[i].replace('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/1D/FITS/',''))    
        else:
            spec =glob('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/1D/FITS/*N*%s*' % ID)
            if len(spec) >= 1:
                spec_n = []
                for i in range(len(spec)):
                    spec_n.append(spec[i].replace('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/1D/FITS/','')) 
                    
        if not os.path.exists(direct):
            os.makedirs(direct)
        
        if len(two_d) >= 1:
            for i in range(len(two_d_n)):
                copyfile(two_d[i], direct + '/' + two_d_n[i])
                
        if len(spec) >= 1:
            for i in range(len(spec)):
                copyfile(spec[i], direct + '/' + spec_n[i])
        
        for i in range(len(one_d_n)):
            copyfile(one_d[i], direct + '/' + one_d_n[i])
        
        print 'in data'

In [71]:
Get_file_list(obv_dir,'n1184')

in data


In [72]:
for i in range(len(quvjDB.index)):
    print quvjDB.index[i]
    Get_file_list(obv_dir,quvjDB.index[i])

n542


in data
n1184
in data
n2023
in data
n2116


n2265


in data
n2302
in data
n2484


in data
n2684
in data
n2882


...

in data
s45972
in data
s46066


in data
s46345
in data
s46377
s46562


s46846
in data
s46888
s47223


in data
s47276
s47677
in data
s47691


in data
s47873
s48631
in data
s49708
